In [2]:
import os

In [3]:
%pwd

'd:\\Project\\Chest Cancer Classification\\End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Project\\Chest Cancer Classification\\End-to-End-Chest-Cancer-Classification-using-MLflow-and-DVC'

In [6]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [10]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [11]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig) -> PrepareBaseModelConfig:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [12]:

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-07-03 02:35:07,804: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-03 02:35:07,820: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-03 02:35:07,825: INFO: common: created directory at: artifacts]
[2024-07-03 02:35:07,827: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 11s 0us/step
[2024-07-03 02:35:23,761: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                

In [16]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.8 MB 3.1 MB/s eta 0:00:13
   ---------------------------------------- 0.5/38.8 MB 4.7 MB/s eta 0:00:09
    --------------------------------------- 0.7/38.8 MB 5.1 MB/s eta 0:00:08
   - -------------------------------------- 1.1/38.8 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.3/38.8 MB 5.7 MB/s eta 0:00:07
   - -------------------------------------- 1.6/38.8 MB 5.5 MB/s eta 0:00:07
   - -------------------------------------- 1.8/38.8 MB 5.5 MB/s eta 0:00:07
   -- ------------------------------------- 2.1/38.8 MB 5.6 MB/s eta 0:00:07
   -- ------------------------------------- 2.4/38.8 MB 5.7 MB/s eta 0:00:07
   -- ------------------------------------- 2.7/38.8 MB 5.8 MB/s eta 0:00:07
   --- ------------------------------------ 2.9/38.8 MB 5.5 MB/s eta 0:00:07
   --- ------------------------------------ 3.2/38.8 MB 5.5 MB/s eta 0:00:07
   ---